In [22]:
import os
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [89]:
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, train_size=0.66) #, random_state=1)